Import Libraries


In [26]:
import numpy as np
from osgeo import gdal
from scipy import stats
import timeit

<img src="data/thumb.png" alt="Drawing" style="width: 300px;"/>

Load test data

In [5]:
r = gdal.Open("data\\nlcd_2001_landcover_2011_edition_2014_10_sub.tif")
g = np.array(r.GetRasterBand(1).ReadAsArray())

In [6]:
##stats.describe(g) 
out = np.zeros(g.shape,dtype='uint64')
g.shape

(1734, 2501)

Function to create data structure function takes in a grid with mxn and returns a grid with mxnxc where c is the number of classes.  A vectorized version would be faster

In [7]:
def dtFZS(g,out):
    for xw in range(1,g.shape[1]):
        for yw in range(1,g.shape[0]):
            out[yw,xw] = out[yw-1,xw] + g[yw,xw]
    print("done")

In [8]:
##out = dtFZS(g)
DS_FstZS = np.zeros(g.shape,dtype='uint64')
dtFZS(g,DS_FstZS)


done


In [9]:
stats.describe(DS_FstZS)
##out.shape

DescribeResult(nobs=1734, minmax=(array([0, 0, 0, ..., 0, 0, 0], dtype=uint64), array([     0,  90090,  90537, ..., 113531, 112942, 113583], dtype=uint64)), mean=array([     0.        ,  39471.85870819,  39549.93886967, ...,
        55162.80046136,  54622.01153403,  54913.44925029]), variance=array([  0.00000000e+00,   7.43734955e+08,   7.51916039e+08, ...,
         1.12397767e+09,   1.11742125e+09,   1.11700624e+09]), skewness=array([ 0.        ,  0.21883235,  0.21780447, ...,  0.09384659,
        0.09849129,  0.1000836 ]), kurtosis=array([-3.        , -1.23819352, -1.22937554, ..., -1.22189322,
       -1.22273959, -1.20557672]))

Fast Summary Statistics Function
    

In [20]:
def FstZS(v,DS_FstZS):
    total = 0;
    for p in range(1,len(v)):
        ## If moving left
        if v[p][1] - v[p-1][1] > 0:
            total += DS_FstZS[v[p][0],v[p][1]]
        ## If moving Right
        elif v[p][1] - v[p-1][1] < 0:
            total -= DS_FstZS[v[p][0],v[p][1]]
    return int(total)

In [21]:
## Test Data set
z = 1000
## Create test box z in size
v = [(x,1) for x in range(1,z+1)] + [(z,x) for x in range(1+1,z+1)] + [(x,z) for x in range(z-1,0,-1)] + [(1,x) for x in range(z-1,0,-1)]

In [36]:
#g.shape
FstZS(v,DS_FstZS)
#%timeit 'FstZS(v,DS_FstZS)'

47168172

In [50]:
##%timeit 'np.sum(g[1:1001,1:1001])'
np.sum(g[2:1001,2:1001])

47168226

Pretty Close Not sure why these are slightly different